# Structbio2022 - Practical day 03

## Analysis of MD simulations

Document written by [Adrián Diaz](mailto:adrian.diaz@vub.be) & [David Bickel](mailto:david.bickel@vub.be).
   
<br/>

During the last session, we submitted the simulations, which should have run by know.

> #### *In a terminal, go to the `3_production` directory and have a look at the generated files.* ####
>
> * Which file contains the parameters of the simulation?
> * Which file contains structural information?
> * Which file contains the trajectory (coordinates over time)?
>
> Use the following commands to do so:
>
> | BASH command | Function |
> |--------------|----------|
> | `cd <path/to/directory>` | Changes to a given directory |
> | `pwd` | Prints the current working directory |
> | `ls` | Shows the content of a directory |
> | `ls -lh` | Same as above in more detail |
> | `less <file>` | Shows the content of a file, exit by pressing [Q] |


***

Let's have a first look at the energetic properties of system throughout the simulation. This can give us a first impression if something went wrong during the simulation.

> #### *Can you verify that the simulations ran consistently by looking at its energetic properties?* ####
> Which properties would you like to look at? Here is a list of all the availble properties. Feel free to play around.
> (*e.g.*, How does temperature correlate with kinetic/total energy?)
> ```python
> [ 
>   'Time', 'Bond', 'U-B', 'Proper Dih.', 'Improper Dih.', 'CMAP Dih.',
>   'LJ-14', 'Coulomb-14', 'LJ (SR)', 'Coulomb (SR)', 'Coul. recip.',
>   'Potential', 'Kinetic En.', 'Total Energy', 'Conserved En.',
>   'Temperature', 'Pressure', 'Constr. rmsd', 'Box-X', 'Box-Y', 'Box-Z',
>   'Volume', 'Density', 'pV', 'Enthalpy', 'Vir-XX', 'Vir-XY', 'Vir-XZ',
>   'Vir-YX', 'Vir-YY', 'Vir-YZ', 'Vir-ZX', 'Vir-ZY', 'Vir-ZZ', 'Pres-XX',
>   'Pres-XY', 'Pres-XZ', 'Pres-YX', 'Pres-YY', 'Pres-YZ', 'Pres-ZX',
>   'Pres-ZY', 'Pres-ZZ', '#Surf*SurfTen', 'Box-Vel-XX', 'Box-Vel-YY',
>   'Box-Vel-ZZ', 'T-Protein', 'T-non-Protein'
> ]
> ```

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import panedr
%matplotlib inline

# Read the data
data = panedr.edr_to_df("../3_production/npt_pme.edr")

fig, ax = plt.subplots(1,1, figsize=(4,2), dpi=150)
ax.grid(True, "both", "both", linewidth=.5, alpha=.5)

# Plot the data
ax.plot(data["Time"], data['Total Energy'])

ax.set_ylabel("Energy | $kJ \cdot mol^{-1}$")
ax.set_xlabel("Simulation time | ps")
plt.show()

***

Now, lets have a first look at our protein complex throughout the simulation.

> #### *Does this look, how you would expected?* ####
> How would you have expected teh simulations to look like?
> Have a look at the structure and think, what could have happend to it?

In [ ]:
import MDAnalysis as mda
import nglview as nv

md = mda.Universe("../3_production/npt_pme_0001.tpr", "../3_production/npt_pme.xtc", in_memory=True, in_memory_step=200)
view = nv.show_mdanalysis(md, default_representation=False)
view._remote_call("setSize", target="Widget", args=["800px", "600px"])
view.add_cartoon("protein", color_scheme="sstruc")
view.add_representation("line", selection="water")
view.add_representation("ball+stick", selection="ion")
view.center() # Center and zoom molecule
view

***

## Creation of an index file

To facilitate later analysis, let's create an index file.
These files are used, to tell GROMACS, on which atoms certain analyses should be performed.

> #### *Create and index file with groups for each ccdB, and for ccdA.* ####
>
> GROMACS automatically creates some default groups. You can always display all existing groups by pressing [ENTER].
> After you created the groups for the proteins, you can rename them, *e.g.*, `name 19 ccdB_1` renames group #19 to "ccdB_1".
> Use the following names:
> * ccdB_1
> * ccdB_2
> * ccdA

## Unwrapping the coordinates, stripping solvent

To get a nice representation of the trajectory, we need to:
* center the protein(s) in the unit cell
* move all atoms into the same unit cell
* doing the above, maintain molecules intact

Now, let's have a look at the trajectory again.

In [ ]:
import MDAnalysis as mda
import nglview as nv

md = mda.Universe("dry_md.tpr", "dry_md.xtc")
view = nv.show_mdanalysis(md, default_representation=False)
view._remote_call("setSize", target="Widget", args=["800px", "600px"])
view.add_cartoon("protein", color_scheme="sstruc")
view.center() # Center and zoom molecule
view

Ok, now we can start with a few standard analyses

***

## RMSD analysis

The RMSD (root mean square deviation) corresponds to the distance between two structures in the conformational space. It is widely applied to compare structures.

$$ RMSD = \sqrt { \frac{1}{N} \sum^{N}_{i=1} (x_i(t) - x_{i, ref})^2 } $$


> #### *Run the RMSD calculation for all proteins, as well as each individual protein.* ####
> Have a look at the results.
> How rigid is a single ccdB in comparison to the whole complex?
> How rigid is ccdB in comparison to ccdA.

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib inline

# Read data
data = pd.read_table("rmsd_complex.xvg", names=["Time", "RMSD"], sep="\s+")

# Plot data
fig, ax = plt.subplots(1,1, figsize=(4,2), dpi=150)
ax.grid(True, "both", "both", linewidth=.5, alpha=.5)
ax.plot(data["Time"], data['RMSD'])
ax.set_ylabel("RMSD | nm")
ax.set_xlabel("Simulation time | ps")
plt.show()

***

## RMSF analysis

The RMSF (root mean square fluctuation) represents the standard deviation of the atomic positions over time. It is thus a measure, of how flexible or dynamic certain regions of a protein are.

$$ \sqrt { \frac{1}{N} \sum^{N}_{i=1} (x_i - \bar{x})^2 } $$

> #### *Calculate the RMSF for each of the ccdB monomers.* ####
> Plot the ccdB monomers in the same plot and compare the results.

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib inline

# Read data (adjust filenames here)
data1 = pd.read_table("rmsf_ccdB.1.xvg", names=["resid", "RMSF"], sep="\s+")
data2 = pd.read_table("rmsf_ccdB.2.xvg", names=["resid", "RMSF"], sep="\s+")

# Plot data
fig, ax = plt.subplots(1,1, figsize=(4,2), dpi=150)
ax.grid(True, "both", "both", linewidth=.5, alpha=.5)
ax.plot(data1["resid"], data1['RMSF'], linewidth=.7)
ax.plot(data2["resid"].iloc[:101], data2['RMSF'].iloc[:101], linewidth=.7)
ax.set_ylabel("RMSF | nm")
ax.set_xlabel("Residues numbers")
plt.show()

> #### *Calculate the RMSF for the whole complex.* ####
> Let's look at the structure and compare the highlighted regions to the plot before.

In [ ]:
import glob
import nglview as nv

pdbfile = "rmsf_complex.pdb"

view = nv.show_structure_file(pdbfile, default_representation=False)
view._remote_call("setSize", target="Widget", args=["800px", "600px"])
view.add_representation("backbone", selection="protein", color_scheme="bfactor")
#view.add_representation("licorice", selection="protein", color_scheme="bfactor")
view.center() # Center and zoom molecule
view

***

## Radius of gyration 

The radius of gyration it is a simple measure of the globularity and compactness of a protein and can be directly related to SAXS data.

$$ R_{gyr} = \sqrt{ \frac{\sum_{i} m_i \lVert \vec{r}_i \rVert^2}{\sum_{i} m_i} } $$

> #### *Let's calculate the radius of gyration for the protein complex.* ####
> How will the inclusion/exclusion of ccdA influence the radius of gyration?

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy.stats import gaussian_kde
%matplotlib inline

# Read data (adjust filenames here)
data = pd.read_table("rgyr.xvg", names=["Time", "Rgyr", "Rgyr_x", "Rgyr_y", "Rgyr_z"], sep="\s+")
values = data['Rgyr']

# Plot data
fig, (ax0, ax1) = plt.subplots(1,2, figsize=(4,2), dpi=150, sharey=True,
                        gridspec_kw={"width_ratios": [3,1], "wspace": .05})
# Time series
ax0.grid(True, "both", "both", linewidth=.5, alpha=.5)
ax0.plot(data["Time"], values, linewidth=.7)
ax0.set_ylabel(r"$R_{gyr}$ | nm")
ax0.set_xlabel("Time | ns")

# Density distribution
kde   = gaussian_kde(values)
rvals = np.linspace(np.min(values), np.max(values), 100)
ax1.grid(True, "both", "both", linewidth=.5, alpha=.5)
ax1.fill_betweenx(rvals, np.zeros(rvals.shape), kde(rvals))

plt.show()